In [31]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import boto3
from botocore.config import Config

from DataFin import FMPClient, S3Client

import os
import dotenv
import boto3
import json
import pandas as pd
import io

dotenv.load_dotenv()

fmp_api_key = os.getenv('FMP_API_KEY')

s3_bucket = os.getenv('PERSONAL_S3_BUCKET_NAME')
aws_access_key =  os.getenv('PERSONAL_AWS_ACCESS_KEY_ID')
aws_secret_key =  os.getenv('PERSONAL_AWS_SECRET_ACCESS_KEY_ID')


fmp = FMPClient(fmp_api_key)

personal_s3 = S3Client(
    aws_account_access_key = aws_access_key,
    aws_secret_account_access_key = aws_secret_key,
    region = 'us-east-1',
    bucket_name = s3_bucket
)

polygon_aws_access_key = os.getenv('POLYGON_AWS_ACCESS_KEY_ID')
polygon_aws_secret_key = os.getenv('POLYGON_AWS_SECRET_ACCESS_KEY_ID')


session = boto3.Session(
   aws_access_key_id = polygon_aws_access_key,
   aws_secret_access_key = polygon_aws_secret_key
)

s3 = session.client(
   's3',
   endpoint_url='https://files.polygon.io',
   config=Config(signature_version='s3v4'),
)



from datetime import datetime, timedelta
now = datetime.now()
yesterday = now - timedelta(days=1)

year = now.year
month = f'{now.month:02d}'
day = f'{now.day:02d}'
yesterday = f'{yesterday.day:02d}'
print(month, day, yesterday)


bucket_name = 'flatfiles'
object_key = f'us_stocks_sip/minute_aggs_v1/2025/{month}/2025-{month}-{yesterday}.csv.gz'

obj = s3.get_object(Bucket=bucket_name, Key=object_key)


df = pd.read_csv(io.BytesIO(obj['Body'].read()), compression='gzip')


df

04 16 15


,ticker,volume,open,close,high,low,window_start,transactions
0,A,26288,104.300,104.585,104.855,104.300000,1744723800000000000,75
1,A,5433,104.585,104.290,104.585,104.100000,1744723860000000000,128
2,A,920,104.470,104.470,104.470,104.470000,1744723920000000000,29
3,A,429,105.070,104.860,105.070,104.860000,1744723980000000000,12
4,A,1008,104.700,104.875,104.875,104.700000,1744724040000000000,33
...,...,...,...,...,...,...,...,...
1598237,ZYXI,7547,2.205,2.210,2.210,2.201121,1744747140000000000,89
1598238,ZYXI,506,2.200,2.200,2.200,2.200000,1744747200000000000,2
1598239,ZYXI,100,2.200,2.200,2.200,2.200000,1744757940000000000,1
1598240,ZZZ,374,24.640,24.640,24.640,24.640000,1744728360000000000,5
